In [ ]:
%matplotlib nbagg
import numpy as np
import atomap.api as am
import hyperspy.api as hs
import atomap.dummy_data as dummy_data
import matplotlib.pyplot as plt

s = hs.load('/Users/usuario/Python PM Office/EPFL/20221028/L_A4-5_45/DF Stack_06 aligned sum using FFT spots Inverse.dm3') #introduce first inverse ABF or HAADF
s.plot()

#get A positions
A_positions = am.get_atom_positions(s, separation=25)
sublattice_A = am.Sublattice(A_positions, image=s.data, color='r')
atom_positions_new = am.add_atoms_with_gui(s, A_positions)


In [ ]:
atom_positions_select = am.select_atoms_with_gui(s, atom_positions_new)

In [ ]:
atom_positions_new = am.add_atoms_with_gui(s, atom_positions_select)

In [ ]:
sublattice_A = am.Sublattice(atom_positions_new, s)
sublattice_A.find_nearest_neighbors()
sublattice_A.refine_atom_positions_using_center_of_mass(percent_to_nn=0.5)
sublattice_A.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.3)
sublattice_A.construct_zone_axes(atom_plane_tolerance=0.5)
sublattice_A.plot()
sublattice_A.plot_planes()
sublattice_A.get_position_history().plot()
atom_positions_A = np.array(sublattice_A.atom_positions)
#print("Atom Positions_A-site:")
#for pos in atom_positions_A:
#    print(pos)


In [ ]:
zone_axis_001 = sublattice_A.zones_axis_average_distances[1] #1 introduces vertical zone axes
B_positions = sublattice_A.find_missing_atoms_from_zone_vector(zone_axis_001)

from atomap.tools import remove_atoms_from_image_using_2d_gaussian
image_without_A = remove_atoms_from_image_using_2d_gaussian(sublattice_A.image, sublattice_A,percent_to_nn=0.4)
substracted = hs.signals.Signal2D(image_without_A)
substracted.plot()

In [ ]:
sublattice_B = am.Sublattice(B_positions, image_without_A, color='blue')
sublattice_B.construct_zone_axes(atom_plane_tolerance=0.5)
sublattice_B.refine_atom_positions_using_center_of_mass(percent_to_nn=0.4)
sublattice_B.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.2)
sublattice_B.plot_planes()
atom_lattice = am.Atom_Lattice(image=s.data, name='test', sublattice_list=[sublattice_A, sublattice_B])
atom_lattice.plot()
sublattice_B.get_position_history().plot()
atom_positions_B = np.array(sublattice_B.atom_positions)
#print("Atom Positions_B-site:")
#for pos in atom_positions_B:
#    print(pos)


In [ ]:
s_ABF = hs.load('/Users/usuario/Python PM Office/EPFL/20221028/L_A4-5_45/DF Stack_06 aligned sum using FFT spots.dm3')
s_ABF.plot()

In [ ]:
initial_positions = sublattice_A.atom_positions
sublattice_A2 = am.Sublattice(initial_positions, image=np.divide(1, s_ABF.data), color='r')
atom_positions_new = am.add_atoms_with_gui(s_ABF, initial_positions )

In [ ]:
sublattice_A2 = am.Sublattice(atom_positions_new, image=np.divide(1, s_ABF.data))
sublattice_A2.find_nearest_neighbors()
sublattice_A2.refine_atom_positions_using_center_of_mass()
sublattice_A2.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.4)
sublattice_A2.construct_zone_axes()
image_without_A2 = remove_atoms_from_image_using_2d_gaussian(sublattice_A2.image, sublattice_A2, percent_to_nn=0.4)
substracted = hs.signals.Signal2D(image_without_A2)
substracted.plot()

In [ ]:
initial_positions = sublattice_B.atom_positions
sublattice_B2 = am.Sublattice(initial_positions, image=image_without_A2, color='b')
atom_positions_new = am.add_atoms_with_gui(s_ABF, initial_positions )

In [ ]:
sublattice_B2 = am.Sublattice(atom_positions_new,image_without_A2) #here it should be image_without_A2 but there is a problem with the code and removes the O positions
sublattice_B2.find_nearest_neighbors()
sublattice_B2.refine_atom_positions_using_center_of_mass()
sublattice_B2.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.25)
sublattice_B2.construct_zone_axes(atom_plane_tolerance=0.9)
sublattice_B2.plot()
sublattice_B2.plot_planes()

In [ ]:
zone_axis_002 = sublattice_B2.zones_axis_average_distances[0]# 0 since the oxygen is parallel to B2, Change to 4 or 5 if O is very shifted
image_without_AB = remove_atoms_from_image_using_2d_gaussian(sublattice_B2.image, sublattice_B2, percent_to_nn=0.25)
substracted = hs.signals.Signal2D(image_without_AB)
substracted.plot()

In [ ]:
O_positions = sublattice_B2.find_missing_atoms_from_zone_vector(zone_axis_002,vector_fraction=0.5,extend_outer_edges=False, outer_edge_limit=5) # vector fractiom: fraction of the distance between adjacent atoms (<0.5: left and >0.5: right) 
sublattice_O = am.Sublattice(O_positions, image_without_AB, color='g')
atom_positions_new = am.add_atoms_with_gui(image_without_AB, O_positions) #instead of s I can use image without_AB however sometimes the quality of this image is bad

In [ ]:
sublattice_O = am.Sublattice(atom_positions_new, image=np.divide(1, s_ABF.data), color='g')
sublattice_O.construct_zone_axes(atom_plane_tolerance=0.8)
sublattice_O.refine_atom_positions_using_center_of_mass(percent_to_nn=0.3)
sublattice_O.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.2)

sublattice_O.get_position_history().plot()

atom_positions_O = np.array(sublattice_O.atom_positions)
#print("Atom Positions:")
#for pos in atom_positions_O:
#    print(pos)


In [ ]:
atom_lattice = am.Atom_Lattice(image=s_ABF.data, name='ABO3', sublattice_list=[sublattice_A, sublattice_B, sublattice_O])
atom_lattice.plot()
#atom_lattice.plot(image=s.data)
#sublattice_O.plot_planes()


In [ ]:
import temul.api as tml
sublattice_A = atom_lattice.sublattice_list[0]
sublattice_O = atom_lattice.sublattice_list[2]
sublattice_A.construct_zone_axes()
za0 = sublattice_A.zones_axis_average_distances[1]
za1 = sublattice_A.zones_axis_average_distances[0]
sublattice_A.plot_planes(image=atom_lattice.image)# use this function to reveal the planes and select the appropriate ones. The number of the plane is indicated on the top
s_p = sublattice_A.get_polarization_from_second_sublattice(za0, za1, sublattice_O)
vector_list = s_p.metadata.vector_list
x, y = [i[0] for i in vector_list], [i[1] for i in vector_list]
u, v = [i[2] for i in vector_list], [i[3] for i in vector_list]
sampling, units = 0.008902054, 'nm' #image calibration
tml.plot_polarisation_vectors(x, y, u, v, image=atom_lattice.image,sampling=sampling, units=units,unit_vector=False, save=None, scalebar=True, plot_style='vector', color='r', overlay=True, monitor_dpi=100)

In [ ]:
'''#this gives a file with polarization starting from the bottom line going tot the top line by line

import os

#directory_path = '/Users/usuario/Python PM Office/UCL/20230505/L_c22033b_04/'
#filename = 'DF Stack_03_Interface aligned sum Try ABSF Filtered_polarization.txt'

sublattice_A = atom_lattice.sublattice_list[0]
sublattice_O = atom_lattice.sublattice_list[2]
sublattice_A.construct_zone_axes()
za0 = sublattice_A.zones_axis_average_distances[1]
za1 = sublattice_A.zones_axis_average_distances[0]
s_p = sublattice_A.get_polarization_from_second_sublattice(za0, za1, sublattice_O)
w=vector_list = s_p.metadata.vector_list


rows=21 #number of Oxygen rows.
columns=30  #number of Oxygen columns 

def colsort():
    def sortFirst(val):
        return val[1]
  
    # sort each row individually using the second key 
    d = columns
    p = 0
    for i in range(rows):
        s = w[p:d] # select all the elements of the row
        s.sort(key=sortFirst, reverse=False) # sort the elements from left to right of the row in ascending order
        w[p:d] = s # replace the elements of the list with the sorted ones
        p = d
        d = d + columns
    
    # sort the columns using the first key
    w.sort(key=sortSecond, reverse=True) # sort in ascending order

# Open the file for writing in the specified directory
with open(os.path.join(directory_path, filename), 'w') as file:
    # Write each element of the list to a new line in the file
    for element in w:
        file.write(str(element).replace(',','\t').replace(')','') .replace('(','')  + '\n')

# Close the file
file.close()

In [ ]:
#this gives two files with the angles ordered depending on y value and not ordered.

import os

z1 = sublattice_O.zones_axis_average_distances[0]
z2 = sublattice_O.zones_axis_average_distances[1]
x, y, a = sublattice_O.get_atom_angles_from_zone_vector(z1, z2, degrees=True)
s_angle = sublattice_O.get_property_map(x, y, a, add_zero_value_sublattice=None, upscale_map=1)
s_angle.plot()

#directory_path = '/Users/usuario/Python PM Office/UCL/20230505/L_c22033b_04/'
#filename = 'DF Stack_03_Interface aligned sum Try ABSF Filtered_angle.txt'
#ordered_filename = 'DF Stack_03_Interface aligned sum Try ABSF Filtered_ordered.txt'


# Open the file for writing in the specified directory
with open(os.path.join(directory_path, filename), 'w') as file:
    # Iterate over the lists in parallel and write each row to the file separated by tabs
    for row in zip(x, y, a):
        file.write('\t'.join(map(str, row)) + '\n')

# Close the file
file.close()

# Combine x, y, and a into a list of tuples
data = list(zip(x, y, a))

# Sort the list of tuples based on the second column (y values)
ordered_data = sorted(data, key=lambda tup: tup[1])

# Open the ordered file for writing in the specified directory
with open(os.path.join(directory_path, ordered_filename), 'w') as ordered_file:
    # Iterate over the ordered data and write each row to the file separated by tabs
    for row in ordered_data:
        ordered_file.write('\t'.join(map(str, row)) + '\n')

# Close the ordered file
ordered_file.close()



In [ ]:
'''#orders files which weren not ordred before
import os

# Define the directory path and filenames
#directory_path = '/Users/usuario/Python PM Office/UCL/20230505/L_c220_04/'
#filename = 'DF Stack_04_Interface aligned sum Try_angles.txt'
#ordered_filename = 'DF Stack_04_Interface aligned sum Try_ordered.txt'

# Read the ordered file
data = []
# Open the file for reading in the specified directory
with open(os.path.join(directory_path, filename), 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Split the line into columns (assuming tab-separated values)
        x, y, a = map(float, line.strip().split('\t'))
        # Add the data as a tuple to the list
        data.append((x, y, a))
        
# Sort the list of tuples based on the second column (y values)
ordered_data = sorted(data, key=lambda tup: tup[1])

# Open the ordered file for writing in the specified directory
with open(os.path.join(directory_path, ordered_filename), 'w') as ordered_file:
    # Iterate over the ordered data and write each row to the file separated by tabs
    for row in ordered_data:
        ordered_file.write('\t'.join(map(str, row)) + '\n')

# No need to explicitly close the ordered file since it's automatically closed when exiting the 'with' block




In [ ]:
'''####this gives the averages between bonding angle per line
import os
import pandas as pd
import plotly.express as px

###directory_path = '/Users/usuario/Python PM Office/UCL/20230505/L_c22033b_04/'
##ordered_filename = 'DF Stack_03_Interface aligned sum Try ABSF Filtered_ordered.txt'
#bodning_angle_file = 'DF Stack_03_Interface aligned sum Try ABSF Filtered_bonding angle.txt'

# Read the ordered file
data = []
with open(os.path.join(directory_path, ordered_filename), 'r') as ordered_file:
    # Iterate over each line in the file
    for line in ordered_file:
        # Split the line into columns (assuming tab-separated values)
        x, y, a = map(float, line.strip().split('\t'))
        # Add the data as a tuple to the list
        data.append((x, y, a))

# Calculate averages for each series
averages = []
series = []
prev_x = None
prev_a = None

max_x = max(data, key=lambda x: x[0])[0]  # (max_x/2) is entered to recognise when to change line

for x, y, a in data:
    if prev_x is None:
        prev_x = x
        prev_a = a
        series.append(a)
    elif x < prev_x+50: #+- 50 is entered to avoid changing (< must be used when x in the ordered file starts from large pixel value and vice versa)
        series.append(a)         
        prev_x = x
        prev_a = a
    elif x > prev_x+(max_x/1.5):  #(> must be used when x in the ordered file starts from large pixel value and vice versa)
        avg = sum(series) / len(series)
        averages.append(avg)
        series = []
        prev_x = None
        prev_a = None

# Calculate average for the last series
if series:
    avg = sum(series) / len(series)
    averages.append(avg)

# Calculate differences and subtract from 180
differences = []
for i in range(0, len(averages)-1, 1):
    diff = 180 - abs(averages[i] - averages[i+1])
    differences.append(diff)

# Print the differences
for i, diff in enumerate(differences, start=1):
    print(f"Difference for series {i}: {diff}")

# Create a DataFrame with the differences
#df_differences = pd.DataFrame({'Unit Cell': range(len(differences),0,- 1), 'Difference': differences})
df_differences = pd.DataFrame({'Unit Cell': range(len(differences) - 1,-1,-1), 'Difference': differences})

# Plot the differences using plotly
fig = px.line(df_differences, y='Unit Cell', x='Difference', title='Bonding angles (starting from the substrate)')
fig.update_layout(xaxis_title='Bodning angle (deg)', yaxis_title='Unit Cell')
fig.update_layout(
    xaxis_title='Angle [deg]',
    yaxis_title='Unit cells',
    height=800,  # Specify the desired height
    width=600    # Specify the desired width
)
fig.update_xaxes(dtick=2)
fig.update_yaxes(dtick=5)

# Display the plot inline in Jupyter Notebook
fig.show()    

# Write the differences to a file
with open(os.path.join(directory_path, bodning_angle_file), 'w') as bodning_angle_file:
    for diff in differences:
        bodning_angle_file.write(str(diff) + '\n')


In [ ]:
# Orders the polarization file (needed for the dipole filter below)
import temul.api as tml
sublattice_A = atom_lattice.sublattice_list[0]
sublattice_O = atom_lattice.sublattice_list[2]
sublattice_A.construct_zone_axes()
za0 = sublattice_A.zones_axis_average_distances[1]
za1 = sublattice_A.zones_axis_average_distances[0]
s_p = sublattice_A.get_polarization_from_second_sublattice(za0, za1, sublattice_O)
w=vector_list = s_p.metadata.vector_list


rows=20#number of Oxygen rows.
columns=30  #number of Oxygen columns

# function to return the second element of the
def colsort():
    def sortSecond(val):
        return val[1] 
  
        #sort each column individualy. 
        # using the second key 
    
    
    
    d=rows
    p=0
    w.sort() #sort the list to the first element of each item which is the row.
    for i in range(columns): #number of columns
        s=w[p:d] #Select all the elemnts of the first column
        s.sort(key = sortSecond) #Sort the element from the top to bottom of the column
        w[p:d]=s # replace the elements of the list with the sorted ones
        p=p+rows
        d=d+rows
colsort()
w

In [ ]:
#**********Dipole filter**********
colsort()
x, y = [i[0] for i in w], [i[1] for i in w]
u, v = [i[2] for i in w], [i[3] for i in w]

#print(len(u)) #check the length of the vector so as to create a list with the same size

elements=rows*columns # number of Oxygen atoms

resolution=0.008902054

#the list below is used to interate all over the vectors in the u and v lists.
list_n = [*range(elements)]

#bottom multiplication
w.append(w.pop(0)) # puts the first value of the list at the end of it. This process used so as to multiply the first vector with the one below.
u = [0.2*u[t]+0.1*i[2] for i,t in zip(w,list_n)]
v = [0.2*v[t]+0.1*i[3] for i,t in zip(w,list_n)]

#up multiplication
colsort()
for b in range(elements-1):
    w.append(w.pop(0))#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
u = [u[t]+0.1*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.1*i[3] for i,t in zip(w,list_n)]

#side +1 column
colsort()
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.2*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.2*i[3] for i,t in zip(w,list_n)]

#side +2 column
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.1*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.1*i[3] for i,t in zip(w,list_n)]

#side -1 column
colsort()
for b in range(elements-rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.2*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.2*i[3] for i,t in zip(w,list_n)]

#side -2 column
colsort()
for b in range(elements-rows*2):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.1*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.1*i[3] for i,t in zip(w,list_n)]

#print(w)  #use this function to check if the columns are sepparated correctly
#for diagonal multiplication a combination between bottom and side multiplication is needed. Specifically "for b in range(n+1)"

sampling, units =  resolution, 'nm' #image calibration
tml.plot_polarisation_vectors(x, y, u, v, image=atom_lattice.image,sampling=sampling, units=units,unit_vector=False, save=None, scalebar=True, plot_style='vector', color='r', overlay=True, monitor_dpi=100)

tml.plot_polarisation_vectors(x, y, u, v, image=atom_lattice.image,
                          plot_style="polar_colorwheel",
                          unit_vector=False, overlay=False,
                          save=None, monitor_dpi=200)

tml.plot_polarisation_vectors(x, y, u, v, image=atom_lattice.image, remove_vectors=True,
                          unit_vector=True, plot_style='contour',
                          overlay=True, pivot='middle', save=False,
                          cmap='cet_colorwheel', alpha=0.5,
                          monitor_dpi=200, vector_rep="angle",
                          antialiased=True, degrees=True)


In [ ]:
#**********Averages displacement and stdv per unit cell**********

# filtered averaging
import statistics
import numpy
def mean(data):
    """Return the sample arithmetic mean of data."""
    n = len(data)
    if n < 1:
        raise ValueError('mean requires at least one data point')
    return sum(data)/n # in Python 2 use sum(data)/float(n)

def _ss(data):
    """Return sum of square deviations of sequence data."""
    c = mean(data)
    ss = sum((x-c)**2 for x in data)
    return ss
def stddev(data, ddof=0):
    """Calculates the population standard deviation
    by default; specify ddof=1 to compute the sample
    standard deviation."""
    n = len(data)
    if n < 2:
        raise ValueError('variance requires at least two data points')
    ss = _ss(data)
    pvar = ss/(n-ddof)
    return pvar**0.5

rows=16#number of Oxygen rows.
columns=30  #number of Oxygen columns
resolution=0.008902054

'''
#Here I use the filter to get better averages values
colsort()        

x, y = [i[0] for i in w], [i[1] for i in w]
u, v = [i[2] for i in w], [i[3] for i in w]

#print(len(u)) #check the length of the vector so as to create a list with the same size


#the list below is used to interate all over the vectors in the u and v lists.
list_n = [*range(elements)]

#bottom multiplication
w.append(w.pop(0)) # puts the first value of the list at the end of it. This process used so as to multiply the first vector with the one below.
u = [0.5*u[t]+0*i[2] for i,t in zip(w,list_n)]
v = [0.5*v[t]+0*i[3] for i,t in zip(w,list_n)]

#up multiplication
colsort()
for b in range(elements-1):
    w.append(w.pop(0))#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
u = [u[t]+0*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0*i[3] for i,t in zip(w,list_n)]

#side +1 column
colsort()
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.2*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.2*i[3] for i,t in zip(w,list_n)]

#side +2 column
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.05*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.05*i[3] for i,t in zip(w,list_n)]

#side -1 column
colsort()
for b in range(elements-rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.2*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.2*i[3] for i,t in zip(w,list_n)]

#side -2 column
colsort()
for b in range(elements-rows*2):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.05*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.05*i[3] for i,t in zip(w,list_n)]

#End of the filter
'''


g=0;
c=0;
stdvp=list=[]  #list for stdv of horizontal displacement
stdvq=list=[]  #list for stdv of vertical displacement



print("        Δx                 Δy               STDVx              STDVy")
for q in range(rows): #this range is the number of rows
    t=q
    for i in range(columns): #this range is the number of columns
        b=u[t]
        z=v[t]
        c=c+b
        g=g+z
        t=t+rows     #this range is the number of rows
        stdvp.append(b*1000* resolution)
        stdvq.append(z*1000* resolution)
    
       # stdvq.append(z)
   ##  STDVp=numpy.std(arrp, axis=0)
   # arrq = numpy.array([q])
  #  e=numpy.std(arrq, axis=0)
    STDVp=stddev(stdvp)
    STDVq=stddev(stdvq)

    l=(1000*c*resolution)/columns  #horizontal x average value in pm
    a=(1000*g*resolution)/columns  #and vertical y average value in pm
    print(l, a, STDVp , STDVq ) 
    c=0;
    g=0;      
    stdvp=list=[]  #list for stdv of horizontal displacement
    stdvq=list=[]  #list for stdv of vertical displacement
t=0    